In [ ]:
import pandas as pd
import sqlalchemy, os
from binance import BinanceSocketManager
from binance.client import Client
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.environ.get("API_KEY")
API_SECRET = os.environ.get("API_SECRET")

client = Client(API_KEY, API_SECRET)

In [ ]:
# fetch 1 minute klines for the last day up until now
klines = client.get_historical_klines(
    "BNBBTC", Client.KLINE_INTERVAL_1MINUTE, "1 day ago UTC"
)

# fetch 30 minute klines for the last month of 2017
klines = client.get_historical_klines(
    "ETHBTC", Client.KLINE_INTERVAL_30MINUTE, "1 Dec, 2017", "1 Jan, 2018"
)

# fetch weekly klines since it listed
klines = client.get_historical_klines(
    "NEOBTC", Client.KLINE_INTERVAL_1WEEK, "1 Jan, 2017"
)

# [
#     1499040000000,      // 0 Open time
#     "0.01634790",       // 1 Open
#     "0.80000000",       // 2 High
#     "0.01575800",       // 3 Low
#     "0.01577100",       // 4 Close
#     "148976.11427815",  // 5 Volume
#     1499644799999,      // 6 Close time
#     "2434.19055334",    // 7 Quote asset volume
#     308,                // 8 Number of trades
#     "1756.87402397",    // 9 Taker buy base asset volume
#     "28.46694368",      // 10 Taker buy quote asset volume
#     "17928899.62484339" // 11 Ignore.
# ]

In [ ]:
# The following code was in code cell no 2
# Creating Socket Manager
bsm = BinanceSocketManager(client)

# Creating a socket for BTC-USDT pair
# socket = bsm.trade_socket('BTCUSDT')
socket = bsm.trade_socket(PAIR)

# SQLite EngineSQLite Engine
engine = sqlalchemy.create_engine("sqlite:///SLPUSDTstream.db")

### Function for creating dataframe

In [ ]:
def createframe(msg):
    df = pd.DataFrame([msg])
    df = df.loc[:, ["s", "E", "p"]]
    df.columns = ["symbol", "Time", "Price"]
    df.Price = df.Price.astype(float)
    df.Time = pd.to_datetime(df.Time, unit="ms")
    return df

In [ ]:
while True:
    await socket.__aenter__()
    msg = await socket.recv()
    while msg["e"] == "error":
        msg = await socket.recv()
    frame = createframe(msg)
    frame.to_sql("SLPUSDT", engine, if_exists="append", index=False)
    print(frame)

In [ ]:
msg = await socket.recv()
if msg["e"]:
    print(msg["e"])
print(msg)

In [ ]:
info = client.get_all_tickers()

In [ ]:
ls = {}
for item in info:
    ls[item['symbol']] = item['price']
print(ls['GASBTC'])

In [ ]:
import os
x = {1:'1',2:'2'}
print(x.keys())
clear = lambda : os.system('clear')
clear()
print('hello')

In [56]:
import sqlite3

db = sqlite3.connect('crypto.db')

cursor = db.cursor()

cursor.execute("""
    CREATE TABLE inventory (
        symbol TEXT,
        avg_price REAL
    )""")
db.commit()
db.close()

In [6]:
db = sqlite3.connect('crypto.db')
cursor = db.cursor()
cursor.execute("INSERT INTO inventory VALUES ('BTCUSDT',30000.24)")
db.commit()
db.close()

In [42]:
def fetchall(selection='*'):
    db = sqlite3.connect('crypto.db')
    cursor = db.cursor()
    cursor.execute(f"SELECT {selection} FROM inventory")
    alldata = cursor.fetchall()
    # print(alldata)
    db.commit()
    db.close()
    return alldata

In [11]:
data = [
    ('OMUSDT', 352.53),
    ('ENSUSDT',12.43)
]
db = sqlite3.connect('crypto.db')
cursor = db.cursor()
print(cursor.executemany("INSERT INTO inventory VALUES (?,?)",data))
db.commit()
db.close()

In [49]:
example = [item[0] for item in fetchall('avg_price')]
print(example)

[30000.24, 352.53, 12.43]


In [37]:
i = 1
dictionary = {}
db = sqlite3.connect('crypto.db')
cursor = db.cursor()
cursor.execute(f"SELECT * FROM inventory")
alldata = cursor.fetchall()
symbols = [x[i] for x in alldata]
print(symbols)
for data in alldata:
    print(data[0])
db.commit()
db.close()

[30000.24, 352.53, 12.43]
BTCUSDT
OMUSDT
ENSUSDT


In [57]:
def fetchall():
    db = sqlite3.connect('crypto.db')
    cursor = db.cursor()
    cursor.execute("SELECT * FROM inventory")
    alldata = cursor.fetchall()
    db.commit()
    db.close()
    return alldata

def extract_for_tuple(alldata, i = 0):
    symbols = [x[i] for x in alldata]
    return symbols

def get_buy_price(symbol):
    db = sqlite3.connect('crypto.db')
    cursor = db.cursor()
    cursor.execute(f"SELECT avg_price FROM inventory WHERE symbol = '{symbol}'")
    price = cursor.fetchone()[0]
    db.commit()
    db.close()
    return price

print(extract_for_tuple(fetchall(),1))
dictionary = {}
# for price in 
print(get_buy_price('BTCUSDT'))

[0.9770508, 0.05983786, 1.11238222, 262.30077921, 0.13032534, 0.00574102, 0.58647918, 0.00570771]


TypeError: 'NoneType' object is not subscriptable

In [4]:
def filter_symbol(level, list_of_symbols, threshold, window):
    from datetime import datetime, timedelta

    selected_symbols = []

    for symbol in list_of_symbols:

        klines = client.get_historical_klines(
            symbol,
            Client.KLINE_INTERVAL_1MINUTE,
            str(datetime.now() - timedelta(hours=window)),
            str(datetime.now()),
        )

        percent_change = get_percent_change(klines[0], klines[-1])

        if percent_change > threshold:
            selected_symbols.append(symbol)
            print(symbol, percent_change)
            
    with open("pairs_level_"+str(level)+".txt", "w") as file:
        file.write(str(selected_symbols)[1:-1].replace('\'','').replace(' ',''))
        
    return selected_symbols

import time
import pandas as pd
import sqlalchemy, os
from binance import BinanceSocketManager
from binance.client import Client
from dotenv import load_dotenv
from datetime import datetime, timedelta

load_dotenv()

API_KEY = os.environ.get("API_KEY")
API_SECRET = os.environ.get("API_SECRET")

# Binance Client
client = Client(API_KEY, API_SECRET)

In [11]:
info = client.get_ticker()
print(info)

[{'symbol': 'ETHBTC', 'priceChange': '0.00054700', 'priceChangePercent': '0.926', 'weightedAvgPrice': '0.05949314', 'prevClosePrice': '0.05910200', 'lastPrice': '0.05964900', 'lastQty': '3.45730000', 'bidPrice': '0.05964900', 'bidQty': '9.93450000', 'askPrice': '0.05965000', 'askQty': '19.07000000', 'openPrice': '0.05910200', 'highPrice': '0.06006500', 'lowPrice': '0.05891200', 'volume': '181580.81450000', 'quoteVolume': '10802.81357297', 'openTime': 1654253942036, 'closeTime': 1654340342036, 'firstId': 343828800, 'lastId': 344035127, 'count': 206328}, {'symbol': 'LTCBTC', 'priceChange': '0.00002300', 'priceChangePercent': '1.100', 'weightedAvgPrice': '0.00209987', 'prevClosePrice': '0.00208900', 'lastPrice': '0.00211300', 'lastQty': '18.30300000', 'bidPrice': '0.00211300', 'bidQty': '49.57900000', 'askPrice': '0.00211400', 'askQty': '91.76100000', 'openPrice': '0.00209000', 'highPrice': '0.00212000', 'lowPrice': '0.00207700', 'volume': '91699.92500000', 'quoteVolume': '192.55777616', 

In [7]:
klines = client.get_historical_klines(
            'BTCBUSD',
            Client.KLINE_INTERVAL_1MINUTE,
            str(datetime.now() - timedelta(hours=1)),
            str(datetime.now()),
        )
print(klines)

[]
